# Run the three-layer IRM and define with/without autocorrelation

When running deterministically, we want to whack gamma up high (infinity would be ideal, but linear algebra error results).

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd

from fair21.earth_params import seconds_per_year, earth_radius
from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)

In [ ]:
models = df['model'].unique()
models

## effect of gamma on determininstic forcing

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]
        params[model][run]['stochastic_run'] = False
        params[model][run]['seed'] = 100

In [ ]:
time = np.arange(1850, 2101)
time.shape

In [ ]:
df_forcing = pd.read_csv(
    os.path.join("..", "data", "forcing", "RFMIP-ERF-tier2.csv")
)

In [ ]:
giss_params = params['GISS-E2-1-G']['r1i1p1f1']
giss_params['gamma_autocorrelation']

In [ ]:
ebm = EnergyBalanceModel(**params['GISS-E2-1-G']['r1i1p1f1'])

In [ ]:
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)

In [ ]:
#ebm.stochastic_run

In [ ]:
ebm.run()

In [ ]:
temperature = {}
forcing = {}

In [ ]:
temperature['gamma_2.89'] = ebm.temperature[:,0]
forcing['gamma_2.89'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 0.01
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_0.01'] = ebm.temperature[:,0]
forcing['gamma_0.01'] = ebm.stochastic_forcing

In [ ]:
giss_params

In [ ]:
giss_params['gamma_autocorrelation'] = 1
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_1'] = ebm.temperature[:,0]
forcing['gamma_1'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 10
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_10'] = ebm.temperature[:,0]
forcing['gamma_10'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 100
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_100'] = ebm.temperature[:,0]
forcing['gamma_100'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 1000
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_1000'] = ebm.temperature[:,0]
forcing['gamma_1000'] = ebm.stochastic_forcing

In [ ]:
time = np.arange(1850.5, 2101)

In [ ]:
for gam in ['0.01', '1', '2.89', '10', '100']:
    pl.plot(time, temperature['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('K relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 temperature change')
pl.legend()

In [ ]:
for gam in ['0.01', '1', '2.89', '10', '100', '1000']:
    pl.plot(time, forcing['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('W/m2 relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 forcing')
pl.legend()

In [ ]:
for gam in ['0.01', '1', '2.89', '10', '100', '1000']:
    pl.plot(time, temperature['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('K relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 temperature change')
pl.legend()
pl.xlim(1850,1860)
pl.ylim(-0.05,0.05)

In [ ]:
for gam in ['0.01', '1', '2.89', '10', '100', '1000']:
    pl.plot(time, forcing['gamma_%s' % gam], label='gamma = %s' % gam)
pl.plot(time+1, df_forcing['GISS-E2-1-G TOT'].values, color='k', label='RFMIP')
pl.ylabel('W/m2 relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 forcing')
pl.legend()
pl.xlim(1850,1860)
pl.ylim(-0.5,0.5)

## determine effect on stochastic forcing

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]
        params[model][run]['stochastic_run'] = True
        params[model][run]['seed'] = 100

In [ ]:
time = np.arange(1850, 2101)
time.shape

In [ ]:
df_forcing = pd.read_csv(
    os.path.join("..", "data", "forcing", "RFMIP-ERF-tier2.csv")
)

In [ ]:
giss_params = params['GISS-E2-1-G']['r1i1p1f1']
giss_params['gamma_autocorrelation']

In [ ]:
ebm = EnergyBalanceModel(**params['GISS-E2-1-G']['r1i1p1f1'])

In [ ]:
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)

In [ ]:
#ebm.stochastic_run

In [ ]:
ebm.run()

In [ ]:
temperature = {}
forcing = {}

In [ ]:
temperature['gamma_2.89'] = ebm.temperature[:,0]
forcing['gamma_2.89'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 0.01
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_0.01'] = ebm.temperature[:,0]
forcing['gamma_0.01'] = ebm.stochastic_forcing

In [ ]:
giss_params

In [ ]:
giss_params['gamma_autocorrelation'] = 1
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_1'] = ebm.temperature[:,0]
forcing['gamma_1'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 10
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_10'] = ebm.temperature[:,0]
forcing['gamma_10'] = ebm.stochastic_forcing

In [ ]:
giss_params['gamma_autocorrelation'] = 100
ebm = EnergyBalanceModel(**giss_params)
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
ebm.run()
temperature['gamma_100'] = ebm.temperature[:,0]
forcing['gamma_100'] = ebm.stochastic_forcing

In [ ]:
# giss_params['gamma_autocorrelation'] = 1000
# ebm = EnergyBalanceModel(**giss_params)
# ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)
# ebm.run()
# temperature['gamma_1000'] = ebm.temperature[:,0]
# forcing['gamma_1000'] = ebm.stochastic_forcing

In [ ]:
time = np.arange(1850.5, 2101)

In [ ]:
for gam in ['0.01', '1', '2.89', '10']:
    pl.plot(time, temperature['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('K relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 temperature change')
pl.legend()

In [ ]:
for gam in ['0.01', '1', '2.89', '10']:
    pl.plot(time, forcing['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('W/m2 relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 forcing')
pl.legend()

In [ ]:
for gam in ['0.01', '1', '2.89', '10']:
    pl.plot(time, temperature['gamma_%s' % gam], label='gamma = %s' % gam)
pl.ylabel('K relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 temperature change')
pl.legend()
pl.xlim(1850,1860)
pl.ylim(-0.05,0.05)

In [ ]:
for gam in ['0.01', '1', '2.89', '10']:
    pl.plot(time, forcing['gamma_%s' % gam], label='gamma = %s' % gam)
pl.plot(time+1, df_forcing['GISS-E2-1-G TOT'].values, color='k', label='RFMIP')
pl.ylabel('W/m2 relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 forcing')
pl.legend()
pl.xlim(1850,1860)
pl.ylim(-0.5,0.5)